In [1]:
print("start")

start


In [2]:
import argparse
import tqdm
from pathlib import Path
import multiprocessing
from multiprocessing import Pool

import sys
sys.path.append("../cod_tools/")
from CODFile import CODFile

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<unsigned int, std::allocator<unsigned int> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<double, std::allocator<double> > already registered; second conversion method ignored.
/net/cci/cschlick/miniconda3/envs/restraintsML/envs/cod_tools/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# blacklist = ["/dev/shm/cschlick/COD/cif/4/32/31/4323119.cif",
#              "/dev/shm/cschlick/COD/cif/4/32/31/4323190.cif",
#              "/dev/shm/cschlick/COD/cif/4/32/31/4323118.cif",
#              "/dev/shm/cschlick/COD/cif/4/32/31/4323117.cif",
#               "/dev/shm/cschlick/COD/cif/4/32/31/4323189.cif"# pickling errors?
            # ]

blacklist = []

In [4]:
def parallel_map(worker,work, n_workers):

  pool = Pool(processes=nproc)

  try:
      return list(tqdm.tqdm(pool.imap(worker,work),
                          total=len(work),
                          disable=disable_progress,
                          desc="nproc="+str(nproc)))

  except Exception as  e:
      print('got exception: %r:, terminating the pool' % (e,))
     # p.terminate()
      err = e
      err = None

  if err is not None:
      raise err

In [ ]:


#parser = argparse.ArgumentParser(
                    # prog = 'COD filter',
                    # description = 'Filter a folder of COD files',
                    # epilog = '')
#parser.add_argument('--folder', type=str,help="Path to COD folder")
#args = parser.parse_args()
args = argparse.Namespace()
args.folder = "/dev/shm/cschlick/COD"



cod_dir = Path(args.folder)
cif_files = [path for path in cod_dir.glob("**/*") if path.suffix == ".cif"]
print("Number cif files:",len(cif_files))


#cif_files = cif_files[214063:214063+10000]
work = cif_files

### DO WITH MP

# cif_files = [f for f in cif_files if str(f) not in blacklist]
# print("Number cif files (after blacklist):",len(cif_files))

def worker(file):
  #print(file)
  try:
    codfile =  CODFile(file,cod_id=file.stem)
    return codfile
  except:
    return None

nproc=64
disable_progress = False
results = parallel_map(worker,work,nproc)
# with Pool(processes=nproc) as pool:
#   for result in tqdm.tqdm(pool.map(worker, work),
#                           total=len(work),
#                           disable=disable_progress,
#                           desc="nproc="+str(nproc)):
#       results.append(result)
# del pool



### DO WITHOUT MP

# results = []
# for file in tqdm.tqdm(work):
#     codfile =  CODFile(file,cod_id=file.stem)
#     results.append(codfile)
    

# Sort CODFile objects depending on how they failed
#results = [result for result in results if result is not None]
failed_codfiles = [codfile for codfile in results if codfile.failed]
failed_messages = [codfile.fail_message for codfile in results if codfile.failed]
kept_codfiles = [codfile for codfile in results if not codfile.failed]
print("Failed:",len(failed_codfiles))
print("Kept:",len(kept_codfiles))


fail_counter = {message:0 for message in results[0].failure_messages.values()}
for message in failed_messages:
  fail_counter[message]+=1
  
print("\nReasons for failure:")
fail_check = 0
for key,value in fail_counter.items():
  if value>0:
    print("  ",key.ljust(60," "),":",value)
    fail_check+=value
#print("Fail check:",fail_check)


print("Files passing filters:")
for cod_file in kept_codfiles:
  #print(cod_file.filepath)
  pass


Number cif files: 467767


nproc=64:  65%|█████████████████████████████████████████████████████████▌                               | 302843/467767 [09:24<05:54, 465.62it/s]

In [ ]:
print("h")